In [68]:
import pandas as pd
df = pd.read_csv("school_shooting.csv")
df['Date'] = pd.to_datetime(df['Date'])

df.tail()


,Date,Death,Injuries,Location,State,School_Name,School_Type,Latitude,Longitude
332,2013-03-21,0,0,Southgate,MI,Davidson Middle School,Middle School,42.206116,-83.204609
333,2013-02-27,0,0,Atlanta,GA,Henry W. Grady High School,High School,33.749099,-84.390185
334,2013-02-13,0,0,San Leandro,CA,Hillside Elementary School,Elementary School,37.724930,-122.156077
335,2013-02-01,0,0,Altanta,GA,Morehouse College,College or University,33.749000,-84.388000
336,2013-01-08,0,0,Fort Myers,FL,Apostolic Revival Center Christian School,Elementary School,26.640628,-81.872308


In [69]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
Base = declarative_base()

engine = create_engine("sqlite:///school.sqlite")
conn = engine.connect()

In [71]:
class school_shoot(Base):
    __tablename__ = "School_shooting"
    id = Column(Integer, primary_key=True)
    Date = Column(String) # for late using the date column, we need to use pandas to convert it back to datetime format for analysis 
    Death = Column(Integer)
    Injuries = Column(Integer)
    Location= Column(String)
    State = Column(String)
    School_Name = Column(String)
    School_Type = Column(String)
    Latitude= Column(Float)
    Longitude = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, Date  = {self.Date}, Death ={self.Death}, Injuries = {self.Injuries}, State = {self.State},Location = {self.Location}, School_Name = {self.School_Name}, School_Type = {self.School_Type}, Latitude = {self.Latitude}, Longitude = {self.Longitude} "

In [72]:
Base.metadata.create_all(engine)

In [73]:
df = pd.read_csv("school_shooting.csv")

In [74]:
school_dict = df.to_dict(orient = 'records')
metadata = MetaData(bind=engine)
metadata.reflect()
table1 = sqlalchemy.Table('School_shooting', metadata, autoload=True)
conn.execute(table1.insert(), school_dict)


In [75]:
conn.execute("select * from School_shooting").fetchall()

[(1, '1/7/13', 1, 0, 'Fort Myers', ' FL', 'Apostolic Revival Center Christian School', 'Preschool', 26.640628000000003, -81.87230840000001),
 (2, '1/10/13', 0, 2, 'Taft', ' CA', 'Taft Union High School', 'High School', 50.9954169, -118.6467639),
 (3, '1/12/13', 0, 1, 'Detroit', ' MI', 'Osborn High School ', 'High School', 42.3486635, -83.0567375),
 (4, '1/15/13', 0, 2, 'St. Louis', ' MO', 'Stevens Institute of Business & Arts', 'College or University', 38.62727329999999, -90.19788890000001),
 (5, '1/15/13', 3, 0, 'Hazard', ' KY', 'Hazard Community and Technical College', 'College or University', 37.24954, -83.19322840000001),
 (6, '1/16/13', 1, 0, 'Chicago', ' IL', 'Chicago State University', 'College or University', 41.8755546, -87.6244212),
 (7, '1/22/13', 0, 3, 'Houston', ' TX', 'Lone Star College North Harris Campus', 'College or University', 29.7589382, -95.36769740000001),
 (8, '1/31/13', 0, 0, 'Phoenix', 'AZ', 'Cesar Chavez High School', 'High School', 33.448586600000006, -112.0

In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
engine = create_engine("sqlite:///school.sqlite")
Base = automap_base()
Base.prepare(engine, reflect= True)
Base.classes.keys()

['School_shooting']

In [3]:
session = Session(engine)
School_shooting = Base.classes.School_shooting

In [4]:
session.query(School_shooting.Date).all

<bound method Query.all of <sqlalchemy.orm.query.Query object at 0x105c78278>>

### need Lat, long info for plotting, and hover tick for city name, state name, school name, death, injury

In [5]:
import pandas as pd
import numpy as np

In [7]:
def getData():
    result = session.query(School_shooting.Date, School_shooting.Location, School_shooting.State, School_shooting.Latitude, 
                       School_shooting.Longitude, School_shooting.School_Name, School_shooting.Death,
                      School_shooting.Injuries, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    
    df2['Year'] = ['20' + str(j).split("/")[2] for j in df2['Date']] 
    return df2
df2 = getData()
df2.head()

,Date,Location,State,Latitude,Longitude,School_Name,Death,Injuries,School_Type,Year
0,1/7/13,Fort Myers,FL,26.640628,-81.872308,Apostolic Revival Center Christian School,1,0,Preschool,2013
1,1/10/13,Taft,CA,50.995417,-118.646764,Taft Union High School,0,2,High School,2013
2,1/12/13,Detroit,MI,42.348664,-83.056737,Osborn High School,0,1,High School,2013
3,1/15/13,St. Louis,MO,38.627273,-90.197889,Stevens Institute of Business & Arts,0,2,College or University,2013
4,1/15/13,Hazard,KY,37.249540,-83.193228,Hazard Community and Technical College,3,0,College or University,2013


In [37]:
type(np.asscalar(df2["Death"][0]))

int

In [128]:
df2['Date'] = pd.to_datetime(df2['Date'])
mask = df2["Year"] == str(2014)
yr = df2[mask]
yr['Date'] = [str(j).split(" ")[0] for j in yr['Date']]
df = yr
df
names = list(df.columns)
Outerlst = []
for i in range(len(df)):
    innerList = []
    for j in range(len(names)):
        innerList.append(df.iloc[i, names.index(names[j])])
        #print(innerList)
    Outerlst.append(dict(zip(names, innerList)))
for index in range(len(Outerlst)):
    
    for key, value in Outerlst[index].items():
        
        if value == str:
            next
        else:
            Outerlst[index][key] = np.asscalar(np.array([value])) 
            


/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


int

In [129]:
def map(year):
    
    df2['Date'] = pd.to_datetime(df2['Date'])
    mask = df2["Year"] == str(year)
    yr = df2[mask]
    yr['Date'] = [str(j).split(" ")[0] for j in yr['Date']]
    df = yr
    df
    names = list(df.columns)
    Outerlst = []
    for i in range(len(df)):
        innerList = []
        for j in range(len(names)):
            innerList.append(df.iloc[i, names.index(names[j])])
            #print(innerList)
        Outerlst.append(dict(zip(names, innerList)))
    for index in range(len(Outerlst)):

        for key, value in Outerlst[index].items():

            if value == str:
                next
            else:
                Outerlst[index][key] = np.asscalar(np.array([value])) 

    
    return Outerlst

In [132]:
type(map(2014)[0]['Latitude'])

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


float

## get year data

In [ ]:
def yearOPtion():
    df2= getData()
    yearList = []
    str_date = [str(date) for date in df2['Date'].dt.year]
    
    yearlst = list(set(str_date))
    yearlst.sort()
    return jsonify(yearlst)

In [ ]:
# get states

yearList = []
for state in df2['State']:
    
    yearList.append(state)

In [ ]:
for title in list(yr13.columns):
    littlelist.append([i for i in yr13[title]])

### getting bar graph data

In [33]:
def getData():
    result = session.query(School_shooting.Date, School_shooting.Location, School_shooting.State, School_shooting.Latitude, 
                       School_shooting.Longitude, School_shooting.School_Name, School_shooting.Death,
                      School_shooting.Injuries, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2["Date"] = pd.to_datetime(df2["Date"])
    df2['Year'] = ['20' + str(j).split("-")[2] for j in df2['Date']] 
    df2['State'] = [i.strip() for i in df2['State']]
    return df2
df2 = getData()
df2 = df2.dropna()
df2.head()

,Date,Location,State,Latitude,Longitude,School_Name,Death,Injuries,School_Type,Year
0,2013-01-07,Fort Myers,FL,26.640628,-81.872308,Apostolic Revival Center Christian School,1,0,Preschool,2007 00:00:00
1,2013-01-10,Taft,CA,50.995417,-118.646764,Taft Union High School,0,2,High School,2010 00:00:00
2,2013-01-12,Detroit,MI,42.348664,-83.056737,Osborn High School,0,1,High School,2012 00:00:00
3,2013-01-15,St. Louis,MO,38.627273,-90.197889,Stevens Institute of Business & Arts,0,2,College or University,2015 00:00:00
4,2013-01-15,Hazard,KY,37.249540,-83.193228,Hazard Community and Technical College,3,0,College or University,2015 00:00:00


In [26]:
df2.groupby(["State",'School_Type']).agg({"School_Name": "count"})

School_Name
State School_Type                       
AL    College or University            3
      High School                      5
      Middle School                    2
AR    College or University            1
      Elementary School                1
      High School                      2
      Middle School                    1
AZ    College or University            1
      High School                      4
CA    College or University            5
      Community College                1
      Elementary School                5
      High School                      5
      Middle School                    2
CO    High School                      4
CT    High School                      1
DE    College or University            2
      Middle School                    1
FL    College or University            8
      Elementary School                6
      High School                      7
      K-12 School                      1
      Preschool                        3
GA    College or University           12
      Elementary School                3
      High School                      7
      Middle School                    3
IA    College or University            1
      High School                      1
      Middle School                    1
...                                  ...
SC    Elementary School                1
      High School                      3
      K-12 School                      1
SD    College or University            1
      High School                      1
TN    College or University            6
      Elementary School                2
      High School                      3
      Middle School                    1
TX    College or University           12
      Community College                3
      Elementary School                3
      High School                     12
      Middle School                    2
UT    College or University            4
      Elementary School                1
      High School                      2
      Middle School                    1
VA    College or University            4
      Elementary School                1
      High School                      1
      Middle School                    1
WA    College or University            3
      Elementary School                2
      High School                      5
WI    College or University            2
      Elementary School                1
      High School                      1
      K-12 School                      1
WV    College or University            1

[127 rows x 1 columns]

In [25]:
df2['State'].unique()

array(['FL', 'CA', 'MI', 'MO', 'KY', 'IL', 'TX', 'AZ', 'GA', 'VA', 'AL',
       'MA', 'MS', 'NC', 'NV', 'PA', 'CO', 'TN', 'CT', 'NM', 'IN', 'SC',
       'LA', 'IA', 'OH', 'WA', 'OR', 'UT', 'OK', 'WI', 'MD', 'SD', 'MT',
       'NY', 'MN', 'AR', 'KS', 'ID', 'WV', 'DE', 'NE', 'ME'], dtype=object)

In [63]:
mask = df2['State'] == "PA"
df = df2[mask]
df
count = df.groupby(['School_Type']).agg({"School_Name": "count"})
count = count.reset_index()
df

,Date,Location,State,Latitude,Longitude,School_Name,Death,Injuries,School_Type,Year
22,2013-11-13,Pittsburgh,PA,40.441694,-79.990086,Brashear High School,0,3,High School,2013 00:00:00
29,2014-01-17,Philadelphia,PA,39.952415,-75.163576,Delaware Valley Charter High School,0,2,High School,2017 00:00:00
30,2014-01-20,Chester,PA,39.849557,-75.355746,Widener University,0,1,College or University,2020 00:00:00
86,2016-01-29,Philadelphia,PA,39.952415,-75.163576,Benjamin Franklin High School,0,0,High School,2029 00:00:00
115,2018-01-31,Philadelphia,PA,39.952415,-75.163576,Lincoln High School,1,0,High School,2031 00:00:00
156,2017-12-13,Monaca,PA,40.687289,-80.271451,Penn State Beaver,0,0,College or University,2013 00:00:00
157,2017-12-12,East Stroudsburg,PA,41.003646,-75.176666,East Stroudsburg University,0,0,College or University,2012 00:00:00
177,2017-09-30,Commodore,PA,40.713121,-78.943365,Purchase Line Elementary School,0,0,Elementary School,2030 00:00:00
282,2015-04-02,Monaca,PA,40.687289,-80.271451,Community College of Beaver County,0,0,College or University,2002 00:00:00
292,2014-12-17,Pittsburgh,PA,40.441694,-79.990086,Sunnyside Elementary School,0,0,Elementary School,2017 00:00:00


In [34]:
df2['School_Type'].unique()

array(['Preschool', 'High School', 'College or University',
       'Middle School', 'Elementary School', 'K-12 School',
       'Community College'], dtype=object)

In [64]:
schoolType =list(df['School_Type'].unique())
schoolType

['High School', 'College or University', 'Elementary School']

In [65]:
School_list = []
for types in schoolType:
    mask = df["School_Type"] == types
    School_list.append(list(df[mask]['School_Name']))
School_list

[['Brashear High School',
  'Delaware Valley Charter High School',
  'Benjamin Franklin High School',
  'Lincoln High School'],
 ['Widener University',
  'Penn State Beaver',
  'East Stroudsburg University',
  'Community College of Beaver County'],
 ['Purchase Line Elementary School',
  'Sunnyside Elementary School',
  'King Elementary School']]

In [67]:
ticks = dict(zip(list(schoolType), School_list))
ticks

{'College or University': ['Widener University',
  'Penn State Beaver',
  'East Stroudsburg University',
  'Community College of Beaver County'],
 'Elementary School': ['Purchase Line Elementary School',
  'Sunnyside Elementary School',
  'King Elementary School'],
 'High School': ['Brashear High School',
  'Delaware Valley Charter High School',
  'Benjamin Franklin High School',
  'Lincoln High School']}

In [72]:
def schoolType(name):
    result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2['State'] = [i.strip() for i in df2['State']]
    mask = df2['State'] == name
    df = df2[mask]
    count = df.groupby(['School_Type']).agg({"School_Name": "count"})
    count = count.reset_index()
    schoolType =list(df['School_Type'].unique())
    School_list = []
    for types in schoolType:
        mask = df["School_Type"] == types
        School_list.append(list(df[mask]['School_Name']))
        
    ticks = dict(zip(list(schoolType), School_list))
    return ticks

In [77]:
schoolType('MN')


{'College or University': ['St. Catherine University',
  'University of St. Thomas',
  'Minnesota State University'],
 'Elementary School': ['Crossroads Elementary School'],
 'Preschool': ['Hand and Hand Montessori']}

In [71]:
len(dictt['College or University'])

1

In [80]:
def State_schoolType():
    df2['State'] = [i.strip() for i in df2['State']]    
    statelst = list(df2['State'].unique())
    statelst.sort()
    stateDict = {}
    for name in statelst:
        stateDict[name] = schoolType(name)
        
    return stateDict 

In [81]:
State_schoolType()

{'AL': {'College or University': ['Stillman College',
   None,
   'Alabama A&M University',
   'Tuskegee University'],
  'High School': ['Vigor High School',
   'Murphy High School',
   'Huffman High School',
   'Greensboro High School',
   'Huffman High School'],
  'Middle School': ['Ossie Ware Mitchell Middle School',
   'Alexander City Middle School']},
 'AR': {'College or University': ['University of Arkansas'],
  'Elementary School': ['Lawson Elementary School'],
  'High School': ['North Little Rock High School',
   'Harmony Grove High School'],
  'Middle School': ['Sulphur Rock Magnet School']},
 'AZ': {'College or University': ['Northern Arizona University - Flagstaff'],
  'High School': ['Cesar Chavez High School',
   'Cesar Chavez High School',
   'Corona del Sol High School',
   'Independence High School']},
 'CA': {'College or University': ['Santa Monica College',
   'Angeles Valley College',
   'University of Southern California',
   'Sacramento City College',
   'Universit

In [78]:
stateDict = {}
for name in statelst:
    stateDict[name] = schoolType(name)

In [79]:
stateDict

{'AL': {'College or University': ['Stillman College',
   None,
   'Alabama A&M University',
   'Tuskegee University'],
  'High School': ['Vigor High School',
   'Murphy High School',
   'Huffman High School',
   'Greensboro High School',
   'Huffman High School'],
  'Middle School': ['Ossie Ware Mitchell Middle School',
   'Alexander City Middle School']},
 'AR': {'College or University': ['University of Arkansas'],
  'Elementary School': ['Lawson Elementary School'],
  'High School': ['North Little Rock High School',
   'Harmony Grove High School'],
  'Middle School': ['Sulphur Rock Magnet School']},
 'AZ': {'College or University': ['Northern Arizona University - Flagstaff'],
  'High School': ['Cesar Chavez High School',
   'Cesar Chavez High School',
   'Corona del Sol High School',
   'Independence High School']},
 'CA': {'College or University': ['Santa Monica College',
   'Angeles Valley College',
   'University of Southern California',
   'Sacramento City College',
   'Universit

In [9]:
def schoolType(name):
        result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
        df2 = pd.DataFrame(result)
        df2 = df2.dropna()
        df2['State'] = [i.strip() for i in df2['State']]
        mask = df2['State'] == name
        df = df2[mask]
        count = df.groupby(['School_Type']).agg({"School_Name": "count"})
        count = count.reset_index()
        schoolType =list(df['School_Type'].unique())
        School_list = []
        for types in schoolType:
            mask = df["School_Type"] == types
            School_list.append(list(df[mask]['School_Name']))
            
        ticks = dict(zip(schoolType, School_list))
        return ticks

In [20]:
def State_schoolType(stateName):
    
    result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2 = df2.dropna()
    df2['State'] = [i.strip() for i in df2['State']]    
    statelst = list(df2['State'].unique())
    statelst.sort()
    stateDict = {}
    for name in statelst:
        stateDict[name] = schoolType(name)
    

    return stateDict[f"{stateName}"]


In [21]:
State_schoolType(PA)

NameError: name 'PA' is not defined

In [19]:
name = 'PA'
print(f"{name}")

PA


## table 1 end point

In [4]:
import pandas as pd
def dictTranform(dict_list):
    df = pd.DataFrame(dict_list)
    names = list(df.columns)
    Outerlst = []
    for i in range(len(df)):
        innerList = []
        for j in range(len(names)):
            innerList.append(df.loc[i, names[j]])
        Outerlst.append(dict(zip(names, innerList)))
    return Outerlst

In [6]:
def schoolType(name):
        result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
        df2 = pd.DataFrame(result)
        df2 = df2.dropna()
        df2['State'] = [i.strip() for i in df2['State']]
        mask = df2['State'] == name
        df = df2[mask]
        count = df.groupby(['School_Type']).agg({"School_Name": "count"})
        count = count.reset_index()
        schoolType =list(df['School_Type'].unique())
        School_list = []
        for types in schoolType:
            mask = df["School_Type"] == types
            School_list.append(list(df[mask]['School_Name']))
            
        ticks = dict(zip(schoolType, School_list))
        return ticks


In [11]:
result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
df2 = pd.DataFrame(result)
df2 = df2.dropna()
df2['State'] = [i.strip() for i in df2['State']]    
statelst = list(df2['State'].unique())
statelst.sort()
stateDict = {}
for name in statelst:
    stateDict[name] = schoolType(name)
target = stateDict['FL']



[3, 8, 6, 7, 1]

In [29]:
keylength = []
length = []
for key, val in target.items():
    keylength.append(key) 
    length.append(len(val))
length.index(max(length))
for key, val in target.items():
    if len(val) != max(length):
        target[key] = target[key] + ["-" for i in range(max(length)-len(val))]

In [31]:
final = dictTranform(target)
final

[{'College or University': 'University of Central Florida',
  'Elementary School': 'Agape Christian Academy',
  'High School': 'West Orange High School',
  'K-12 School': 'North Broward Preparatory School',
  'Preschool': 'Apostolic Revival Center Christian School'},
 {'College or University': 'Eastern Florida State College',
  'Elementary School': 'Franklin County Elementary School',
  'High School': 'Miami Carol City High School',
  'K-12 School': '-',
  'Preschool': 'The Academy of Knowledge Preschool'},
 {'College or University': 'Florida State University',
  'Elementary School': 'Oscar Patterson Elementary School',
  'High School': 'Vanguard High School',
  'K-12 School': '-',
  'Preschool': 'Everglades City School'},
 {'College or University': 'Bethune-Cookman University',
  'Elementary School': 'Rock Crusher Elementary School',
  'High School': 'Marjory Stoneman Douglas High School',
  'K-12 School': '-',
  'Preschool': '-'},
 {'College or University': 'University of West Florid

In [32]:
target

{'College or University': ['University of Central Florida',
  'Eastern Florida State College',
  'Florida State University',
  'Bethune-Cookman University',
  'University of West Florida',
  'University of South Florida',
  'University of Florida',
  'University of Miami'],
 'Elementary School': ['Agape Christian Academy',
  'Franklin County Elementary School',
  'Oscar Patterson Elementary School',
  'Rock Crusher Elementary School',
  'Greenwood Lakes Middle School',
  'Apostolic Revival Center Christian School',
  '-',
  '-'],
 'High School': ['West Orange High School',
  'Miami Carol City High School',
  'Vanguard High School',
  'Marjory Stoneman Douglas High School',
  'Forest High School',
  'Lecanto High School',
  'Alexander W. Dreyfoos School of the Arts',
  '-'],
 'K-12 School': ['North Broward Preparatory School',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-'],
 'Preschool': ['Apostolic Revival Center Christian School',
  'The Academy of Knowledge Preschool',
  'Everglade

# endpooint for table

In [168]:
import pandas as pd
def getData():
    result = session.query(School_shooting.Date, School_shooting.Location, School_shooting.State, School_shooting.Latitude, 
                       School_shooting.Longitude, School_shooting.School_Name, School_shooting.Death,
                      School_shooting.Injuries, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2['Date'] = pd.to_datetime(df2['Date'])
    df2['Date'] = [str(i).split(' ')[0] for i in df2['Date']]
    comboLst = list(zip(df2['Death'][130:], df2["Injuries"][130:]))
    Death = []
    Injury = []
    for j in comboLst:
        if j[0] == 0 and j[1] == 0:
            Death.append("-")
            Injury.append("-")
        else:
            Death.append(j[0])
            Injury.append(j[1])
    df2['Death'] = list(df2['Death'][0:130]) + Death
    df2['State'] = [i.split()[0] for i in df2['State']]
    df2['Injuries'] = list(df2['Injuries'][0:130]) + Injury
    df2['Location'] = [i.upper() for i in df2['Location']] 
    Stype = []
    for i in df2['School_Type']:
        if i == None :
            Stype.append("-")
        else:
            Stype.append(i.upper().strip())

    df2['School_Type'] = Stype
    name = []
    for i in df2['School_Name']:
        if i == None :
            name.append("-")
        else:
            name.append(i.upper().strip())

    df2['School_Name'] = name
    
    nulllist = []
    for i in df2['School_Name']:
        if i == None:
            nulllist.append("-")
        else:
            nulllist.append(i)
    df2['School_Name'] = nulllist
    names = list(df2.columns)
    Outerlst = []
    for i in range(len(df2)):
        innerList = []
        for j in range(len(names)):
            innerList.append(df2.loc[i, names[j]])
        Outerlst.append(dict(zip(names, innerList)))
    
    return Outerlst

In [169]:
getData()

[{'Date': '2013-01-07',
  'Death': 1,
  'Injuries': 0,
  'Latitude': 26.640628000000003,
  'Location': 'FORT MYERS',
  'Longitude': -81.87230840000001,
  'School_Name': 'APOSTOLIC REVIVAL CENTER CHRISTIAN SCHOOL',
  'School_Type': 'PRESCHOOL',
  'State': 'FL'},
 {'Date': '2013-01-10',
  'Death': 0,
  'Injuries': 2,
  'Latitude': 50.9954169,
  'Location': 'TAFT',
  'Longitude': -118.6467639,
  'School_Name': 'TAFT UNION HIGH SCHOOL',
  'School_Type': 'HIGH SCHOOL',
  'State': 'CA'},
 {'Date': '2013-01-12',
  'Death': 0,
  'Injuries': 1,
  'Latitude': 42.3486635,
  'Location': 'DETROIT',
  'Longitude': -83.0567375,
  'School_Name': 'OSBORN HIGH SCHOOL',
  'School_Type': 'HIGH SCHOOL',
  'State': 'MI'},
 {'Date': '2013-01-15',
  'Death': 0,
  'Injuries': 2,
  'Latitude': 38.62727329999999,
  'Location': 'ST. LOUIS',
  'Longitude': -90.19788890000001,
  'School_Name': 'STEVENS INSTITUTE OF BUSINESS & ARTS',
  'School_Type': 'COLLEGE OR UNIVERSITY',
  'State': 'MO'},
 {'Date': '2013-01-15',

In [160]:
df2['School_Type'].isnull().unique()

array([False,  True])

In [166]:
name = []
for i in df2['School_Name']:
    if i == None :
        name.append("-")
    else:
        name.append(i.upper().strip())
    
df2['School_Name'] = name

In [167]:
df2['School_Name']

0        APOSTOLIC REVIVAL CENTER CHRISTIAN SCHOOL
1                           TAFT UNION HIGH SCHOOL
2                               OSBORN HIGH SCHOOL
3             STEVENS INSTITUTE OF BUSINESS & ARTS
4           HAZARD COMMUNITY AND TECHNICAL COLLEGE
5                         CHICAGO STATE UNIVERSITY
6            LONE STAR COLLEGE NORTH HARRIS CAMPUS
7                         CESAR CHAVEZ HIGH SCHOOL
8                              PRICE MIDDLE SCHOOL
9                    UNIVERSITY OF CENTRAL FLORIDA
10                     NEW RIVER COMMUNITY COLLEGE
11                                STILLMAN COLLEGE
12                                             MIT
13               OSSIE WARE MITCHELL MIDDLE SCHOOL
14                            SANTA MONICA COLLEGE
15     RONALD E. MCNAIR DISCOVERY LEARNING ACADEMY
16                        NORTH PANOLA HIGH SCHOOL
17                              CARVER HIGH SCHOOL
18                         AGAPE CHRISTIAN ACADEMY
19                            S

In [152]:
str(type(None))

"<class 'NoneType'>"

In [159]:
for i in df2['School_Name']:
    if i is not None:
        print(i)


APOSTOLIC REVIVAL CENTER CHRISTIAN SCHOOL
TAFT UNION HIGH SCHOOL
OSBORN HIGH SCHOOL 
STEVENS INSTITUTE OF BUSINESS & ARTS
HAZARD COMMUNITY AND TECHNICAL COLLEGE
CHICAGO STATE UNIVERSITY
LONE STAR COLLEGE NORTH HARRIS CAMPUS
CESAR CHAVEZ HIGH SCHOOL
PRICE MIDDLE SCHOOL
UNIVERSITY OF CENTRAL FLORIDA
NEW RIVER COMMUNITY COLLEGE
STILLMAN COLLEGE
MIT
OSSIE WARE MITCHELL MIDDLE SCHOOL
SANTA MONICA COLLEGE
RONALD E. MCNAIR DISCOVERY LEARNING ACADEMY
NORTH PANOLA HIGH SCHOOL
CARVER HIGH SCHOOL
AGAPE CHRISTIAN ACADEMY
SPARKS MIDDLE SCHOOL
NORTH CAROLINA A&T STATE UNIVERSITY
STEPHENSON HIGH SCHOOL
BRASHEAR HIGH SCHOOL
WEST ORANGE HIGH SCHOOL
ARAPAHOE HIGH SCHOOL
EDISON HIGH SCHOOL
LIBERTY TECHNOLOGY MAGNET HIGH SCHOOL
HILLHOUSE HIGH SCHOOL
BERRENDO MIDDLE SCHOOL
DELAWARE VALLEY CHARTER HIGH SCHOOL
WIDENER UNIVERSITY
PURDUE UNIVERSITY
SOUTH CAROLINA STATE UNIVERSITY
ANGELES VALLEY COLLEGE
REBOUND HIGH SCHOOL
TENNESSEE STATE UNIVERSITY
GRAMBLING STATE UNIVERSITY
EASTERN FLORIDA STATE COLLEGE
CESAR